In [17]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import csv
from Bio.SeqRecord import SeqRecord
import torch
import json
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler
import itertools
from collections import Counter

In [18]:
training = pd.read_csv('training_deepfri.csv',index_col=0)
training

,Sequence,Label,Protein
0,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,1,seq1|1|training
1,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,1,seq2|1|training
2,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,1,seq3|1|training
3,MARVGPGRAGVSCQGRGRGRGGSGQRRPPTWEISDSDAEDSAGSEA...,1,seq4|1|training
4,MAAADGGGPGGASVGTEEDGGGVGHRTVYLFDRREKESELGDRPLQ...,1,seq5|1|training
...,...,...,...
2513,MEQGKGLAVLILAIILLQGTLAQSIKGNHLVKVYDYQEDGSVLLTC...,0,seq2516|0|training
2514,MARGPGLAPPPLRLPLLLLVLAAVTGHTAAQDNCTCPTNKMTVCSP...,0,seq2517|0|training
2515,MELSWHVVFIALLSFSCWGSDWESDRNFISTAGPLTNDLLHNLSGL...,0,seq2518|0|training
2516,MWCIVLFSLLAWVYAEPTMYGEILSPNYPQAYPSEVEKSWDIEVPE...,0,seq2519|0|training


In [19]:
testing = pd.read_csv('testing_deepfri.csv',index_col=0)
testing

,Sequence,Label,Protein
0,MAELKSLSGDAYLALSHGYAAAAAGLAYGAAREPEAARGYGTPGPG...,1,sp|Q8NDY6|BHE23_HUMAN
1,MPDPAKSAPAPKKGSKKAVTKAQKKDGKKRKRSRKESYSIYVYKVL...,1,sp|P23527|H2B1O_HUMAN
2,MYQSLALAASPRQAAYADSGSFLHAPGAGSPMFVPPARVPSMLSYL...,1,sp|Q9BWX5|GATA5_HUMAN
3,MNMEGLVMFQDLSIDFSQEEWECLDAAQKDLYRDVMMENYSSLVSL...,1,sp|A8MQ14|ZN850_HUMAN
4,MTRSCSAVGCSTRDTVLSRERGLSFHQFPTDTIQRSKWIRAVNRVD...,1,sp|Q9H5L6|THAP9_HUMAN
...,...,...,...
625,MPLAQLADPWQKMAVESPSDSAENGQQIMDEPMGEEEINPQTEEVS...,0,sp|P51812|KS6A3_HUMAN
626,MAILMLSLQLILLLIPSISHEAHKTSLSSWKHDQDWANVSNMTFSN...,0,sp|Q6UY13|YB003_HUMAN
627,MAQASPPRPERVLGASSPEARPAQEALLLPTGVFQVAEKMEKRTCA...,0,sp|Q9UIL8|PHF11_HUMAN
628,MGKSEGPVGMVESAGRAGQKRPGFLEGGLLLLLLLVTAALVALGVL...,0,sp|Q495T6|MMEL1_HUMAN


In [20]:
def amino_acid_composition(protein_sequence):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    
    count_list = {aa: 0 for aa in amino_acids}
    
    for aa in protein_sequence:
        if aa in count_list:
            count_list[aa] += 1
    total_aa = len(protein_sequence)
    aa_composition = {aa: (count / total_aa) * 100 for aa, count in count_list.items()}

    return aa_composition

In [21]:
def calculate_asdc(protein_sequence, max_skip):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    skip_dipeptides = []
    for i in range(max_skip + 1):
        skip_dipeptides.extend([''.join(pair) for pair in itertools.product(amino_acids, repeat=2)])

    skip_dipeptide_counts = Counter({sd: 0 for sd in skip_dipeptides})
    
    for i in range(len(protein_sequence) - 1):
        for skip in range(max_skip + 1):
            if i + skip + 1 < len(protein_sequence):
                dipeptide = protein_sequence[i] + protein_sequence[i + skip + 1]
                skip_dipeptide_counts[dipeptide] += 1
    
    total_count = sum(skip_dipeptide_counts.values())
    asdc = {sd: count / total_count for sd, count in skip_dipeptide_counts.items()}

    return asdc

In [22]:
def calculate_paac(protein_sequence, lambda_val=2, weight=0.05):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    
    paac = {f'PAAC_{aa}': 0 for aa in amino_acids}
    
    for aa in protein_sequence:
        paac[f'PAAC_{aa}'] += 1
    total_count = len(protein_sequence)
    for aa in amino_acids:
        paac[f'PAAC_{aa}'] /= total_count
    
    sequence_order_effect = [0] * (lambda_val)
    for i in range(1, lambda_val + 1):
        for j in range(len(protein_sequence) - i):
            aa1 = protein_sequence[j]
            aa2 = protein_sequence[j + i]
            sequence_order_effect[i-1] += (ord(aa1) - ord(aa2)) ** 2
        sequence_order_effect[i-1] = sequence_order_effect[i-1] / (len(protein_sequence) - i)
    
    sequence_order_effect = [weight * soe for soe in sequence_order_effect]
    
    for i, soe in enumerate(sequence_order_effect, start=1):
        paac[f'PAAC_SOE_{i}'] = soe
    
    return paac

In [23]:
def calculate_qso(protein_sequence, distance_matrix, weight=0.1, nlag=None):

    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    if nlag is None:
        nlag = len(protein_sequence) - 1

    QSO_vector = np.zeros(20 + nlag)

    norm_occurrence = np.array([protein_sequence.count(aa) for aa in amino_acids], dtype=float)
    norm_occurrence /= norm_occurrence.sum()

    Sk = np.zeros(nlag)
    for k in range(1, nlag + 1):
        for i in range(len(protein_sequence) - k):
            aa1 = amino_acids.index(protein_sequence[i])
            aa2 = amino_acids.index(protein_sequence[i + k])
            Sk[k - 1] += distance_matrix[aa1, aa2]
        if len(protein_sequence) - k != 0: 
            Sk[k - 1] /= (len(protein_sequence) - k)
        else:
            Sk[k - 1] = 0
    denominator = norm_occurrence.sum() + weight * Sk.sum()
    QSO_vector[:20] = norm_occurrence / denominator

    QSO_vector[20:] = (weight * Sk) / denominator

    return QSO_vector

In [24]:
for i in range(len(training)):
    protein_sequence = training['Sequence'][i]
    aac = amino_acid_composition(protein_sequence)
    asdc = calculate_asdc(protein_sequence, 1)
    paac = calculate_paac(protein_sequence)
    
    distance_matrix = np.random.rand(20, 20)
    distance_matrix = (distance_matrix + distance_matrix.T) / 2 
    np.fill_diagonal(distance_matrix, 0) 
    qso_feature_vector = calculate_qso(protein_sequence, distance_matrix, nlag=24)
    
    feature_pro = list(asdc.values()) + list(aac.values()) + list(paac.values()) + list(qso_feature_vector)
    if i == 0:
        df = pd.DataFrame([feature_pro])
    else:
        df.loc[i] = feature_pro

In [25]:
df['Label'] = training['Label']
df['Protein'] = training['Protein']
df

,0,1,2,3,4,5,6,7,8,9,...,478,479,480,481,482,483,484,485,Label,Protein
0,0.004044,0.000578,0.001155,0.006355,0.001155,0.002311,0.000578,0.004044,0.004622,0.003466,...,0.022582,0.022008,0.022566,0.021923,0.022482,0.022042,0.021654,0.021756,1,seq1|1|training
1,0.008448,0.001056,0.002112,0.003168,0.006336,0.007392,0.002112,0.000000,0.003168,0.008448,...,0.021043,0.021945,0.021940,0.020791,0.022025,0.022005,0.022156,0.021358,1,seq2|1|training
2,0.000000,0.003044,0.000000,0.003044,0.000000,0.001522,0.000000,0.000000,0.006088,0.004566,...,0.022013,0.022696,0.022104,0.022511,0.021244,0.022788,0.022270,0.022843,1,seq3|1|training
3,0.021192,0.002649,0.007947,0.011921,0.002649,0.015894,0.000000,0.002649,0.000000,0.015894,...,0.022520,0.022638,0.023043,0.022583,0.022515,0.023156,0.022675,0.022089,1,seq4|1|training
4,0.003494,0.000499,0.002246,0.002246,0.001747,0.003244,0.000998,0.003743,0.001997,0.004742,...,0.021612,0.021368,0.021606,0.021487,0.021651,0.021074,0.021798,0.021159,1,seq5|1|training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,0.005540,0.000000,0.005540,0.005540,0.000000,0.005540,0.000000,0.011080,0.005540,0.002770,...,0.020995,0.021783,0.020555,0.021292,0.022163,0.021293,0.020746,0.021658,0,seq2516|0|training
2514,0.009331,0.000000,0.004666,0.001555,0.001555,0.010886,0.001555,0.001555,0.001555,0.009331,...,0.022598,0.021755,0.023061,0.022641,0.022646,0.021676,0.021757,0.022947,0,seq2517|0|training
2515,0.006232,0.001039,0.005972,0.003635,0.004414,0.004674,0.002856,0.002597,0.002337,0.008310,...,0.021990,0.022297,0.022302,0.021962,0.022121,0.022510,0.022347,0.021900,0,seq2518|0|training
2516,0.006555,0.000000,0.005827,0.001457,0.000728,0.006555,0.001457,0.002185,0.002185,0.005098,...,0.021576,0.021993,0.021628,0.021742,0.022188,0.022378,0.022157,0.022679,0,seq2519|0|training


In [26]:
#df.to_csv("feature_1part_train.csv")

In [27]:
for i in range(len(testing)):
    protein_sequence = testing['Sequence'][i]
    aac = amino_acid_composition(protein_sequence)
    asdc = calculate_asdc(protein_sequence, 1)
    paac = calculate_paac(protein_sequence)
    
    distance_matrix = np.random.rand(20, 20)
    distance_matrix = (distance_matrix + distance_matrix.T) / 2 
    np.fill_diagonal(distance_matrix, 0) 
    qso_feature_vector = calculate_qso(protein_sequence, distance_matrix, nlag=24)
    
    feature_pro = list(asdc.values()) + list(aac.values()) + list(paac.values()) + list(qso_feature_vector)
    if i == 0:
        df = pd.DataFrame([feature_pro])
    else:
        df.loc[i] = feature_pro

In [28]:
df['Label'] = testing['Label']
df['Protein'] = testing['Protein']
df

,0,1,2,3,4,5,6,7,8,9,...,478,479,480,481,482,483,484,485,Label,Protein
0,0.046980,0.002237,0.008949,0.013423,0.008949,0.015660,0.002237,0.002237,0.002237,0.026846,...,0.021871,0.020567,0.021357,0.021676,0.022245,0.020738,0.020500,0.021841,1,sp|Q8NDY6|BHE23_HUMAN
1,0.004016,0.000000,0.000000,0.004016,0.000000,0.008032,0.008032,0.000000,0.016064,0.000000,...,0.021857,0.022046,0.020301,0.020703,0.023088,0.022294,0.020849,0.022437,1,sp|P23527|H2B1O_HUMAN
2,0.018963,0.003793,0.002528,0.000000,0.003793,0.016435,0.003793,0.000000,0.003793,0.012642,...,0.022947,0.022322,0.021846,0.022065,0.022861,0.022996,0.021981,0.023371,1,sp|Q9BWX5|GATA5_HUMAN
3,0.000459,0.000459,0.000000,0.000000,0.006431,0.002297,0.001837,0.004134,0.001837,0.004134,...,0.021838,0.022738,0.021863,0.022616,0.022976,0.023014,0.023086,0.022786,1,sp|A8MQ14|ZN850_HUMAN
4,0.001664,0.000555,0.001109,0.004992,0.002773,0.002773,0.001109,0.001664,0.003328,0.007210,...,0.022477,0.021712,0.022439,0.021670,0.022595,0.022017,0.022608,0.022358,1,sp|Q9H5L6|THAP9_HUMAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,0.004739,0.001354,0.004062,0.004062,0.001354,0.002708,0.001354,0.002031,0.004062,0.007448,...,0.022040,0.021851,0.022477,0.021681,0.021696,0.021625,0.022372,0.022128,0,sp|P51812|KS6A3_HUMAN
626,0.000000,0.000000,0.005348,0.000000,0.000000,0.010695,0.005348,0.010695,0.005348,0.010695,...,0.020632,0.021739,0.022122,0.022526,0.022120,0.022736,0.021283,0.021192,0,sp|Q6UY13|YB003_HUMAN
627,0.004552,0.001517,0.000000,0.006070,0.004552,0.001517,0.003035,0.003035,0.006070,0.007587,...,0.022599,0.023459,0.022218,0.022264,0.023158,0.022969,0.021891,0.021935,0,sp|Q9UIL8|PHF11_HUMAN
628,0.005145,0.000643,0.003859,0.003215,0.003215,0.005145,0.000000,0.001929,0.001929,0.007717,...,0.021790,0.021993,0.021766,0.022198,0.021166,0.021066,0.021985,0.022090,0,sp|Q495T6|MMEL1_HUMAN


In [29]:
#df.to_csv("feature_1part_test.csv")

In [ ]:
########################DeepFRI features##############################

In [90]:
training_features = pd.read_csv('feature_1part_train.csv',index_col=0)
file_path = 'training/DeepFRI_MF_pred_scores.json'

with open(file_path, 'r') as file:
    data = json.load(file)
    
deepfri_train_features = pd.DataFrame(data['Y_hat'])
merged_data = pd.concat([deepfri_train_features, training_features], axis=1)

In [91]:
merged_data = merged_data.drop('Protein',axis=1)
merged_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,Label
0,1.974557e-06,0.000034,0.000179,0.000181,0.001079,0.000141,0.000058,0.001089,0.000030,1.366192e-06,0.010479,0.000045,0.007558,0.000292,0.000025,0.001514,0.004123,0.000103,0.011815,0.001695,0.000302,0.001898,0.000311,0.000103,0.000348,0.000027,0.000145,0.000054,0.000222,0.000370,4.385461e-05,0.000514,0.000032,0.000577,0.002330,0.000139,0.000541,0.000095,0.000991,0.000041,0.000430,9.096012e-05,6.570524e-05,0.000669,0.000963,0.001754,0.000121,0.000271,0.000097,0.000429,0.008009,0.030637,0.009289,0.000522,0.008904,0.000120,0.000310,0.000292,0.007679,0.004361,0.001785,0.000579,3.560086e-05,0.000214,0.000541,0.000235,0.000072,0.000133,0.008921,0.000081,0.003359,0.007062,0.000069,0.000040,0.065854,0.001316,0.001937,0.011174,0.000400,0.000283,0.000146,0.000410,0.000209,0.000111,0.000579,0.000561,0.008498,0.000061,0.000072,0.002172,8.966147e-05,0.000028,7.576808e-05,0.001515,0.000807,0.000220,1.284921e-05,0.000117,0.000104,1.220769e-05,0.000042,0.004382,0.000822,0.000085,0.000593,1.523183e-05,0.000386,0.000016,0.000274,0.000979,0.010035,0.000064,0.000065,0.000118,0.000267,0.006449,0.000099,0.000576,0.000285,0.000876,0.000066,0.000256,2.022122e-05,0.000371,0.000816,0.001193,0.000127,0.000539,0.000157,0.004834,0.000035,0.003159,0.000033,0.001362,0.002267,0.000441,0.000084,0.001722,0.000447,0.000041,0.000428,0.007640,6.773616e-06,0.000058,0.000355,0.000014,0.000872,0.000046,0.050892,0.000215,0.000401,0.000083,0.000045,0.001443,0.000176,0.005802,0.000196,0.000109,0.000669,0.000059,0.000049,0.000292,0.000092,0.016570,0.001982,6.553597e-07,0.004314,0.000181,0.001939,0.000152,0.005892,0.000165,0.000143,0.000150,2.692913e-06,0.000912,0.000182,0.000123,0.000062,0.002830,0.001141,7.940393e-06,0.000139,0.000298,0.000036,0.001893,0.000065,0.000080,0.000797,2.559900e-06,0.000115,0.002326,0.004823,0.000136,0.001279,0.000068,0.000139,0.000180,0.000296,1.075769e-04,0.004742,0.000049,0.001402,0.000147,7.852473e-07,0.000681,3.089687e-05,0.000474,0.005587,0.001516,0.000099,0.000184,0.000054,0.000317,0.000158,0.000208,0.000070,0.000078,0.000174,0.000126,0.000109,0.072652,0.000087,0.000373,0.000

In [59]:
#merged_data.to_csv('training_features.csv')

In [92]:
testing_features = pd.read_csv('feature_1part_test.csv',index_col=0)
file_path = 'testing/DeepFRI_MF_pred_scores.json'

with open(file_path, 'r') as file:
    data = json.load(file)
    
deepfri_test_features = pd.DataFrame(data['Y_hat'])
merged_data = pd.concat([deepfri_test_features, testing_features], axis=1)

In [93]:
merged_data = merged_data.drop('Protein',axis=1)
merged_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,Label
0,4.954666e-05,0.000119,0.000852,0.000947,0.002525,0.000192,0.000362,0.001079,0.000462,2.964692e-05,0.006071,1.581122e-03,0.015466,0.002117,3.635361e-04,0.007051,0.004136,0.000832,0.005558,0.003688,0.001858,0.006373,0.004041,0.001171,0.002404,0.000105,0.000213,0.000203,0.002784,0.001090,0.000768,0.001125,0.000358,0.001913,0.003498,0.000380,0.000849,0.000498,0.004508,0.000523,0.000551,0.000650,0.000380,0.001712,0.024005,0.003965,0.000443,0.000905,0.001750,0.001012,0.010797,0.070541,0.025976,0.002694,0.059340,0.000922,0.001452,0.001746,0.053696,0.012298,0.016460,0.003267,0.000791,0.001023,0.000854,0.001451,0.000333,0.000426,0.019629,0.000288,0.003249,0.026480,0.004682,2.165414e-04,0.024931,0.036548,0.004664,0.003213,0.001614,0.000908,0.000740,0.003829,0.001205,0.000205,0.000838,0.000897,0.009407,0.000372,0.000582,0.008567,0.002256,0.000142,2.611577e-04,0.013373,0.003665,0.000854,3.287074e-04,0.000971,0.000438,1.969766e-04,0.000253,0.014027,0.002211,0.000459,0.002202,8.575261e-05,0.000738,0.000048,0.003081,0.006018,0.022519,0.004568,0.000649,0.000405,0.000535,0.005885,0.000115,0.001637,2.780095e-04,0.003464,5.405346e-04,0.000679,0.000179,0.006222,0.015000,0.002041,1.325594e-03,0.005021,0.001223,0.003890,0.000107,0.002656,0.000162,0.001889,0.004369,0.000223,0.000120,0.001293,0.000410,8.299696e-05,0.000842,0.008304,3.510866e-04,0.000388,0.005772,8.936774e-05,0.002446,0.000687,0.028592,0.000551,0.007967,0.000119,0.000325,0.001632,0.000280,0.062194,0.002101,0.000604,0.001372,0.000372,0.000279,0.000568,0.000848,0.032993,0.016212,3.644411e-06,0.012689,0.000849,0.004698,0.000864,0.009108,0.000555,0.002165,0.000676,3.791032e-05,0.000602,0.000719,0.001117,0.001045,0.012393,0.000856,0.002401,0.002366,0.000811,0.000214,0.005813,4.276312e-04,0.000172,0.001185,4.333627e-05,6.807843e-04,0.011075,0.008564,0.000381,0.005621,0.001066,0.000802,0.001211,0.000360,3.491015e-04,0.025617,0.000364,0.009643,0.002560,4.415647e-06,0.001211,5.842263e-04,0.001249,0.085106,0.001710,0.000767,0.002717,0.000158,0.002116,0.000754,0.001941,0.000781,0.000195,0.000421,0.001125,0.000716,0.030959,0.000996,0.

In [76]:
#merged_data.to_csv('testing_features.csv')

In [82]:
file_path = 'testing/DeepFRI_MF_pred_scores.json'

with open(file_path, 'r') as file:
    data = json.load(file)
    
deepfri_test_features = pd.DataFrame(data['Y_hat'])

file_path = 'training/DeepFRI_MF_pred_scores.json'

with open(file_path, 'r') as file:
    data = json.load(file)
    
deepfri_train_features = pd.DataFrame(data['Y_hat'])

In [84]:
deepfri_test_features['Label'] = testing_features['Label']
deepfri_train_features['Label'] = training_features['Label']

In [87]:
########################Only DeepFRI features##############################
deepfri_test_features.index = pd.RangeIndex(start=2518, stop=3148)
from pycaret.classification import *
s=setup(data=deepfri_train_features, target='Label',test_data=deepfri_test_features, fold = 5)

,Description,Value
0,Session id,7574
1,Target,Label
2,Target type,Binary
3,Original data shape,"(3148, 943)"
4,Transformed data shape,"(3148, 943)"
5,Transformed train set shape,"(2518, 943)"
6,Transformed test set shape,"(630, 943)"
7,Numeric features,942
8,Preprocess,True
9,Imputation type,simple


In [88]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:22:24
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7617,0.8394,0.7752,0.7582,0.7655,0.5234,0.5252,10.0180
rf,Random Forest Classifier,0.7574,0.8369,0.7744,0.7514,0.7617,0.5147,0.5165,0.3800
xgboost,Extreme Gradient Boosting,0.7574,0.8379,0.7633,0.7569,0.7590,0.5147,0.5161,3.8860
et,Extra Trees Classifier,0.7522,0.8317,0.7680,0.7476,0.7563,0.5044,0.5064,0.1880
ada,Ada Boost Classifier,0.7478,0.8151,0.7625,0.7423,0.7516,0.4956,0.4968,1.8460
svm,SVM - Linear Kernel,0.7375,0.0000,0.8205,0.7106,0.7581,0.4749,0.4874,0.0980
lr,Logistic Regression,0.7355,0.8176,0.7705,0.7242,0.7455,0.4710,0.4735,0.4720
ridge,Ridge Classifier,0.7355,0.0000,0.7752,0.7224,0.7466,0.4710,0.4739,0.1180
knn,K Neighbors Classifier,0.6954,0.7755,0.7133,0.6921,0.7014,0.3908,0.3923,0.7260
dt,Decision Tree Classifier,0.6771,0.6771,0.6855,0.6748,0.6800,0.3543,0.3544,0.7880


Processing:   0%|          | 0/69 [00:00<?, ?it/s]


KeyboardInterrupt



In [94]:
######################## AAC, ASDC, PAAC, QSOrder + DeepFRI features##############################
scaled_X_train = pd.read_csv('training_features.csv',index_col=0)
scaled_X_test = pd.read_csv('testing_features.csv',index_col=0)
scaled_X_test.index = pd.RangeIndex(start=2518, stop=3148)

In [80]:
from pycaret.classification import *
s=setup(data=scaled_X_train, target='Label',test_data=scaled_X_test, fold = 5)

,Description,Value
0,Session id,2769
1,Target,Label
2,Target type,Binary
3,Original data shape,"(3148, 1429)"
4,Transformed data shape,"(3148, 1429)"
5,Transformed train set shape,"(2518, 1429)"
6,Transformed test set shape,"(630, 1429)"
7,Numeric features,1428
8,Preprocess,True
9,Imputation type,simple


In [81]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:11:06
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7780,0.8427,0.7752,0.7829,0.7780,0.5560,0.5575,1.1320
gbc,Gradient Boosting Classifier,0.7764,0.8585,0.7800,0.7786,0.7777,0.5528,0.5551,14.4780
rf,Random Forest Classifier,0.7756,0.8477,0.7990,0.7670,0.7810,0.5512,0.5543,0.4520
ridge,Ridge Classifier,0.7737,0.0000,0.7704,0.7795,0.7737,0.5473,0.5490,0.2080
et,Extra Trees Classifier,0.7721,0.8488,0.7919,0.7665,0.7770,0.5441,0.5473,0.2300
xgboost,Extreme Gradient Boosting,0.7697,0.8547,0.7871,0.7642,0.7744,0.5393,0.5411,5.1560
ada,Ada Boost Classifier,0.7439,0.8239,0.7490,0.7436,0.7454,0.4877,0.4888,2.6660
knn,K Neighbors Classifier,0.7192,0.7838,0.7641,0.7021,0.7315,0.4385,0.4405,0.9620
svm,SVM - Linear Kernel,0.7184,0.0000,0.7568,0.7482,0.7275,0.4368,0.4700,0.2380
dt,Decision Tree Classifier,0.6621,0.6621,0.6728,0.6590,0.6658,0.3241,0.3242,1.0400



KeyboardInterrupt



In [168]:
######################## AAC, ASDC, PAAC, QSOrder + Protein Bert + DeepFRI features#####################
temp_train = pd.read_csv('training_features.csv',index_col=0)
temp_test = pd.read_csv('testing_features.csv',index_col=0)
df_label_train = temp_train['Label']
df_label_test = temp_test['Label']

temp_train = temp_train.drop('Label',axis = 1)
temp_test = temp_test.drop('Label',axis = 1)

temp_train.columns = range(1428)
temp_test.columns = range(1428)

proteinBert_train = pd.read_csv('proteinbert_train.csv')
proteinBert_test = pd.read_csv('proteinbert_test.csv')

scaled_X_train = pd.concat([proteinBert_train, temp_train], axis=1)
scaled_X_test = pd.concat([proteinBert_test, temp_test], axis=1)

#scaled_X_train = scaled_X_train.drop('Label',axis = 1)
#scaled_X_test = scaled_X_test.drop('Label',axis = 1)

scaled_X_train.columns = range(2452)
scaled_X_test.columns = range(2452)

scaled_X_train['Label'] = df_label_train
scaled_X_test['Label'] = df_label_test


scaled_X_test.index = pd.RangeIndex(start=2518, stop=3148)

In [169]:
scaled_X_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,2203,2204,2205,2206,2207,2208,2209,2210,2211,2212,2213,2214,2215,2216,2217,2218,2219,2220,2221,2222,2223,2224,2225,2226,2227,2228,2229,2230,2231,2232,2233,2234,2235,2236,2237,2238,2239,2240,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250,2251,2252,2253,2254,2255,2256,2257,2258,2259,2260,2261,2262,2263,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287,2288,2289,2290,2291,2292,2293,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303,2304,2305,2306,2307,2308,2309,2310,2311,2312,2313,2314,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352,2353,2354,2355,2356,2357,2358,2359,2360,2361,2362,2363,2364,2365,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375,2376,2377,2378,2379,2380,2381,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400,2401,2402,2403,2404,2405,2406,2407,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417,2418,2419,2420,2421,2422,2423,2424,2425,2426,2427,2428,2429,2430,2431,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441,2442,2443,2444,2445,2446,2447,2448,2449,2450,2451,Label
2518,-0.208924,0.221001,-0.198544,-0.210953,-0.226499,-0.221386,-0.219529,-0.217138,0.203369,-0.213728,0.215224,0.217973,-0.220173,0.200760,-0.210996,-0.223238,0.213569,0.200829,0.214071,-0.160682,-0.198790,-0.216550,-0.212251,-0.219809,0.179497,-0.218586,0.213158,0.218280,0.210457,0.203781,0.215990,0.215680,0.211156,-0.220416,0.197607,-0.211278,0.219888,-0.214666,0.216445,-0.219240,0.214349,-0.224037,0.185140,-0.216622,-0.216590,-0.219456,-0.215494,0.211274,0.214951,-0.220262,0.204981,-0.220136,0.218944,-0.212224,0.217604,-0.186509,0.208405,0.216281,0.219040,0.223108,0.210898,0.208016,-0.216936,0.219194,0.213895,0.217864,-0.211751,-0.212030,-0.208178,-0.210931,-0.170813,0.214689,0.220624,0.208652,0.213254,-0.208639,-0.216229,0.215522,0.206761,0.217625,-0.216828,-0.212145,-0.221003,0.213531,-0.215694,0.208803,0.221199,0.182568,-0.217115,-0.219022,-0.216157,0.203266,0.208940,0.225935,0.217831,0.183463,-0.212302,-0.219716,0.209407,-0.224298,0.220238,-0.216365,0.217429,0.207623,-0.216487,0.225128,-0.211156,-0.225117,0.225077,-0.207272,-0.214067,-0.218478,-0.177873,-0.225007,0.224729,-0.209857,-0.207918,-0.202561,-0.213078,0.215145,-0.222018,-0.219520,-0.222867,0.160674,-0.221795,-0.212642,-0.215715,0.219128,0.219521,-0.212416,0.213345,0.221201,0.175994,-0.217039,0.214195,-0.210407,0.220750,-0.198559,-0.208205,-0.219734,0.220724,-0.212862,-0.204211,0.217473,0.213609,0.221020,-0.202674,0.205700,0.208652,-0.208806,-0.215706,-0.209182,-0.219876,0.203837,-0.214237,0.185544,-0.214277,-0.207079,0.213473,-0.208954,0.211504,0.214986,-0.211825,0.217710,-0.207043,0.224305,0.217718,-0.218495,0.217202,0.183118,0.211206,-0.203294,0.219122,0.214218,-0.218584,0.207477,0.206998,-0.216909,0.220782,-0.198136,0.214422,0.217064,0.223401,-0.215844,-0.181136,-0.214595,-0.223064,0.187278,0.214163,0.177373,-0.210400,-0.207958,0.204933,0.215260,-0.214

In [170]:
temp_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231,1232,1233,1234,1235,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427
0,1.974557e-06,0.000034,0.000179,0.000181,0.001079,0.000141,0.000058,0.001089,0.000030,1.366192e-06,0.010479,0.000045,0.007558,0.000292,0.000025,0.001514,0.004123,0.000103,0.011815,0.001695,0.000302,0.001898,0.000311,0.000103,0.000348,0.000027,0.000145,0.000054,0.000222,0.000370,4.385461e-05,0.000514,0.000032,0.000577,0.002330,0.000139,0.000541,0.000095,0.000991,0.000041,0.000430,9.096012e-05,6.570524e-05,0.000669,0.000963,0.001754,0.000121,0.000271,0.000097,0.000429,0.008009,0.030637,0.009289,0.000522,0.008904,0.000120,0.000310,0.000292,0.007679,0.004361,0.001785,0.000579,3.560086e-05,0.000214,0.000541,0.000235,0.000072,0.000133,0.008921,0.000081,0.003359,0.007062,0.000069,0.000040,0.065854,0.001316,0.001937,0.011174,0.000400,0.000283,0.000146,0.000410,0.000209,0.000111,0.000579,0.000561,0.008498,0.000061,0.000072,0.002172,8.966147e-05,0.000028,7.576808e-05,0.001515,0.000807,0.000220,1.284921e-05,0.000117,0.000104,1.220769e-05,0.000042,0.004382,0.000822,0.000085,0.000593,1.523183e-05,0.000386,0.000016,0.000274,0.000979,0.010035,0.000064,0.000065,0.000118,0.000267,0.006449,0.000099,0.000576,0.000285,0.000876,0.000066,0.000256,2.022122e-05,0.000371,0.000816,0.001193,0.000127,0.000539,0.000157,0.004834,0.000035,0.003159,0.000033,0.001362,0.002267,0.000441,0.000084,0.001722,0.000447,0.000041,0.000428,0.007640,6.773616e-06,0.000058,0.000355,0.000014,0.000872,0.000046,0.050892,0.000215,0.000401,0.000083,0.000045,0.001443,0.000176,0.005802,0.000196,0.000109,0.000669,0.000059,0.000049,0.000292,0.000092,0.016570,0.001982,6.553597e-07,0.004314,0.000181,0.001939,0.000152,0.005892,0.000165,0.000143,0.000150,2.692913e-06,0.000912,0.000182,0.000123,0.000062,0.002830,0.001141,7.940393e-06,0.000139,0.000298,0.000036,0.001893,0.000065,0.000080,0.000797,2.559900e-06,0.000115,0.002326,0.004823,0.000136,0.001279,0.000068,0.000139,0.000180,0.0

In [166]:
from pycaret.classification import *
s=setup(data=scaled_X_train, target='Label',test_data=scaled_X_test, fold = 5)

,Description,Value
0,Session id,8959
1,Target,Label
2,Target type,Binary
3,Original data shape,"(3148, 2453)"
4,Transformed data shape,"(3148, 2453)"
5,Transformed train set shape,"(2518, 2453)"
6,Transformed test set shape,"(630, 2453)"
7,Numeric features,2452
8,Preprocess,True
9,Imputation type,simple


In [167]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:12:01
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7780,0.8427,0.7752,0.7829,0.7780,0.5560,0.5575,1.6240
ridge,Ridge Classifier,0.7737,0.0000,0.7704,0.7795,0.7737,0.5473,0.5490,0.4800
rf,Random Forest Classifier,0.7709,0.8477,0.7935,0.7627,0.7764,0.5417,0.5441,0.6180
ada,Ada Boost Classifier,0.7439,0.8239,0.7490,0.7436,0.7454,0.4877,0.4888,4.8960
svm,SVM - Linear Kernel,0.7248,0.0000,0.7822,0.7202,0.7377,0.4495,0.4688,0.3960
knn,K Neighbors Classifier,0.7188,0.7839,0.7641,0.7015,0.7312,0.4377,0.4397,1.5280
nb,Naive Bayes,0.6629,0.6693,0.9015,0.6141,0.7291,0.3256,0.3709,0.5680
dt,Decision Tree Classifier,0.6620,0.6620,0.6791,0.6576,0.6679,0.3241,0.3245,1.4860
qda,Quadratic Discriminant Analysis,0.6331,0.6617,0.2979,0.9002,0.4468,0.2662,0.3572,0.6000


Processing:   0%|          | 0/69 [00:00<?, ?it/s]


KeyboardInterrupt

